In [5]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

In [10]:
def compute(delta_eta, eta_max, IC = "Ones", alpha = 0.25):
    # IC = "Born" or IC = "Ones"
    n_steps = round(eta_max / delta_eta)
    print("n steps =", n_steps)
    Nc = 3
    CF = ((Nc ** 2) - 1) / (2 * Nc)
    coef = ((alpha ** 2) * CF / (2 * Nc)) * np.pi * np.sqrt(2 * np.pi / (alpha * Nc)) * delta_eta
    coef_oam = ((alpha**2)*(CF**2) * np.pi)/(4*Nc)
    
    if IC == "Ones":
        G = np.ones((n_steps + 1, n_steps + 1))
        G2 = np.ones((n_steps + 1, n_steps + 1))
        Gm = np.ones((n_steps + 1, n_steps + 1, n_steps + 1))
        Gm2 = np.ones((n_steps + 1, n_steps + 1, n_steps + 1))
        
        Gm3 = np.ones((n_steps + 1, n_steps + 1, n_steps + 1))
        I3 = np.ones((n_steps + 1, n_steps + 1))
        I4 = np.ones((n_steps + 1, n_steps + 1))
        I5 = np.ones((n_steps + 1, n_steps + 1))
        I6 = np.ones((n_steps + 1, n_steps + 1))
    else:
        G = np.array([[coef * ((2 * i) + ((CF - 2) * j)) for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gm = np.array([[[coef * ((2 * i) + ((CF - 2) * j)) for j in range(n_steps + 1)] for k in range(n_steps + 1)] for i in range(n_steps + 1)])
        G2 = np.array([[coef * i for j in range(n_steps + 1)] for i in range(n_steps + 1)])
        Gm2 = np.array([[[coef * i for j in range(n_steps + 1)] for k in range(n_steps + 1)] for i in range(n_steps + 1)])
        
        I3 = np.array([[coef for j in range(n_steps + 1)] for i in range(n_steps + 1)], dtype='longdouble')
        I4 = np.array([[0 for j in range(n_steps + 1)] for i in range(n_steps + 1)], dtype='longdouble')
        I5 = np.array([[0 for j in range(n_steps + 1)] for i in range(n_steps + 1)], dtype='longdouble')
        I6 = np.array([[0 for j in range(n_steps + 1)] for i in range(n_steps + 1)], dtype='longdouble')
        Gm3 = np.array([[[coef for j in range(n_steps + 1)] for k in range(n_steps + 1)] for i in range(n_steps + 1)], dtype='longdouble')
    
    fac = ((CF - 2) * coef)
    d = (delta_eta ** 2)
    
    for j in range(1, n_steps + 1):
        for i in range(j + 1):
            G[i, j] = G[i, j-1] + fac
            G2[i, j] = G2[i, j-1] 
            I3[i, j] = I3[i, j-1] 
            I4[i, j] = I4[i, j-1] 
            I5[i, j] = I5[i, j-1] 
            I6[i, j] = I6[i, j-1] 
            
            Gm2[i, i, j] = G2[i, j]
            Gm[i, i, j] = G[i, j]
            Gm3[i, i, j] = I3[i, j]
            
            # dipole UV loop
            for ii in range(i, j):
                G[i, j] += d*(Gm[i, ii, j-1] + 3*G[ii, j-1] + 2*G2[ii, j-1] + 2*Gm2[i, ii, j-1])
                I3[i, j] += d*(Gm3[i, ii, j-1] - Gm2[i, ii, j-1])
                I6[i,j] += 0.5*d*(-2*Gm2[i, ii, j-1])
            
            for k in range(i+1, j+1):
                # neighbor IR loop
                Gm2[i, k, j] = Gm2[i, k-1, j-1]
                
                Gm3[i, k, j] = Gm3[i, k-1, j-1]
                
                # neighbor UV loop
                Gm[i, k, j] = Gm[i, k-1, j-1] + fac
                for ii in range(k-1, j):
                    Gm[i, k, j] += d * (Gm[i, ii, j-1] + 3*G[ii, j-1] + 2*G2[ii, j-1] + 2*Gm2[i, ii, j-1])
                    
                    Gm3[i, k, j] += d * (Gm3[i, ii, j-1] - Gm2[i, ii, j-1])
            
            # dipole IR loop
            for ii in range(i):
                G2[i, j] += 2*d*(G[ii, ii + j - i] + 2*G2[ii, ii + j - i])
                
                I3[i, j] += d*(2*I3[ii, ii + j - i] 
                                - 2*G[ii, ii + j - i] - 2*G2[ii, ii + j - i]
                              - 3*I6[ii, ii + j - i] + I5[ii, ii + j - i])
                I4[i,j] += 0.5*d*(2*G[ii, ii + j - i] - 2*I3[ii, ii + j - i] + 9*I5[ii, ii + j - i]
                                    -7*I6[ii, ii + j - i] +5*G2[ii, ii + j - i] + 10*I4[ii, ii + j - i])
                I5[i,j] += 0.5*d*(3*I5[ii, ii + j - i] + 3*G2[ii, ii + j - i] - 3*I6[ii, ii + j - i] 
                                     + 2*I4[ii, ii + j - i] + 2*G[ii, ii + j - i] - 2*I3[ii, ii + j - i])
                I6[i,j] += 0.5*d*(-2*G[ii, ii + j - i] + 2*I3[ii, ii + j - i] - 4*G2[ii, ii + j - i] 
                                      + 4*I6[ii, ii + j - i])
                  
    
        if(np.mod(j*delta_eta, 1) == 0): print(np.round(j * delta_eta, 4))
         
    eps = 0.0000001
    logG = np.log(np.abs(G) + eps)
    logGm = np.log(np.abs(Gm) + eps)
    
    logG2 = np.log(np.abs(G2) + eps)
    logGm2 = np.log(np.abs(Gm2) + eps)
    
    logI3 = np.log(np.abs(I3) + eps)
    logGm3 = np.log(np.abs(Gm3) + eps)
    
    logI4 = np.log(np.abs(I4) + eps)
    logI5 = np.log(np.abs(I5) + eps)
    logI6 = np.log(np.abs(I6) + eps)
    
    outVec = [logG, logG2, logI3, logI4, logI5, logI6, logGm, logGm2]
    
    return outVec

In [12]:
out = compute(0.1, 10)

np.savetxt("test1_G.dat", out[0])
np.savetxt("test1_G2.dat", out[1])
np.savetxt("test1_I3.dat", out[2])
np.savetxt("test1_I4.dat", out[3])
np.savetxt("test1_I5.dat", out[4])
np.savetxt("test1_I6.dat", out[5])

n steps = 100
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
